In [ ]:
import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

from spf.dataset.spf_dataset import v5spfdataset

ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_10_03_38_21_nRX2_rx_circle.zarr"
ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_15_11_44_13_nRX2_bounce.zarr"

import pickle
import os


output_prefix = "./" + os.path.basename(ds_fn) + "_"
full_p = pickle.load(open("full_p.pkl", "rb"))["full_p"]

nthetas = 65
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache="/home/mouse9911/precompute_cache_chunk16",
    paired=True,
    skip_signal_matrix=True,
)

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]

offsets = [
    ds.yaml_config["receivers"][0]["theta-in-pis"] * np.pi,
    ds.yaml_config["receivers"][1]["theta-in-pis"] * np.pi,
]


def resample_from_index(particles, weights, indexes):
    # print(indexes)
    particles[:] = particles[indexes]
    # weights[:] = weights[indexes]
    # add noise to the new samples
    noise = np.random.randn(*particles.shape)
    # noise[:, 0] *= 0.01
    # noise[:, 1] *= 0.01
    noise[:, 0] *= 0.1
    noise[:, 1] *= 0.01
    change_mask = indexes[:-1] == indexes[1:]
    particles[1:][change_mask] += noise[1:][change_mask]
    # weights.resize(len(particles))
    weights.fill(1.0 / len(weights))
    # weights[1:][change_mask] *= 0.1
    weights /= sum(weights)  # normalize


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 2))
    particles[:, 0] = mean[0] + (np.random.randn(N) * std[0])
    particles[:, 1] = mean[1] + (np.random.randn(N) * std[1])
    return particles


def predict(particles, dt=1.0):
    particles[:, 0] += dt * particles[:, 1]
    # N = len(particles)
    # particles[:, 0] += np.random.randn(particles.shape[0]) * 0.0001
    # particles[:, 1] += np.random.randn(particles.shape[0]) * 0.0001


def fix_particles(particles):
    while np.abs(particles[:, 0]).max() > np.pi / 2:
        mask = np.abs(particles[:, 0]) > np.pi / 2
        particles[mask, 0] = np.sign(particles[mask, 0]) * np.pi - particles[mask, 0]
        particles[mask, 1] *= -1
    return particles


def hx_pf(x):
    return np.sin(x[:, 0]) * antenna_spacing * 2 * np.pi / wavelength


def theta_phi_to_bins(theta_phi, nbins):
    if isinstance(theta_phi, float):
        return int(nbins * (theta_phi + np.pi) / (2 * np.pi)) % nbins
    return (nbins * (theta_phi + np.pi) / (2 * np.pi)).astype(int) % nbins


def theta_phi_to_p_vec(thetas, phis, full_p):
    theta_bin = theta_phi_to_bins(thetas, nbins=full_p.shape[0])
    phi_bin = theta_phi_to_bins(phis, nbins=full_p.shape[1])
    return np.take(full_p[:, phi_bin], theta_bin)


def theta_phi_to_p(theta, phi, full_p):
    theta_bins = full_p.shape[0]
    phi_bins = full_p.shape[1]
    theta_bin = int(theta_bins * (theta + np.pi) / (2 * np.pi)) % theta_bins
    phi_bin = int(phi_bins * (phi + np.pi) / (2 * np.pi)) % phi_bins
    return full_p[theta_bin, phi_bin]


def update(particles, weights, z):
    # non-vectorized
    # weights = weights.copy()
    # for idx in range(weights.shape[0]):
    #     weights[idx] *= theta_phi_to_p(particles[idx, 0], z, full_p=full_p)
    # vectorized
    weights *= theta_phi_to_p_vec(particles[:, 0], z, full_p=full_p)
    # assert np.isclose(_weights, weights).all()
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def estimate(particles, weights):
    mean = np.average(particles, weights=weights, axis=0)
    var = np.average((particles - mean) ** 2, weights=weights, axis=0)
    return mean, var


def neff(weights):
    return 1.0 / np.sum(np.square(weights))


def pf_trajectory_for_phi(rx_idx, ds, N=128):
    particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.1), N=N)
    weights = np.ones((N,)) / N
    trajectory = []
    thetas = []
    vs = []
    for idx in range(len(ds)):
        # particles = fix_particles(particles)
        predict(particles=particles, dt=0.1)
        particles = fix_particles(particles)
        z = np.array(ds[idx][rx_idx]["mean_phase_segmentation"])
        update(particles=particles, weights=weights, z=z)
        particles = fix_particles(particles)
        # resample if too few effective particles
        if neff(weights) < N / 2:
            # print("RESAMPLE")
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            # assert np.allclose(weights, 1 / N)
        mu, var = estimate(particles, weights)
        # particles = create_gaussian_particles(mean=mu, std=np.sqrt(var), N=N)
        trajectory.append({"theta": mu[0], "var": var[0]})
        for particle in particles:
            thetas.append((idx, particle[0]))
            vs.append((idx, particle[1]))
    return trajectory, thetas, vs

In [ ]:
traj1, theta1, vs1 = pf_trajectory_for_phi(1, ds, 128 * 4)

In [ ]:
import matplotlib.pyplot as plt

x = [p[0] for p in theta1]
y = [p[1] for p in theta1]
plt.scatter(x, y, s=1.0, alpha=0.5)

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(2, 2, figsize=(10, 10))

for rx_idx in [0, 1]:
    ax[1, rx_idx].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    ax[1, rx_idx].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    trajectory, thetas, vs = pf_trajectory_for_phi(rx_idx, ds, N=128)
    n = len(trajectory)
    ax[0, rx_idx].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[0, rx_idx].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
    ax[1, rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
        label=f"r{rx_idx} gt theta",
    )
    reduced_gt_theta = np.array(
        [
            reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
            for idx in range(min(n, len(ds)))
        ]
    )

    xs = np.array([x["theta"] for x in trajectory])
    stds = np.sqrt(np.array([x["var"] for x in trajectory]))

    ax[1, rx_idx].fill_between(
        np.arange(xs.shape[0]),
        xs - stds,
        xs + stds,
        label="PF-std",
        color="red",
        alpha=0.2,
    )
    ax[1, rx_idx].scatter(range(len(ds)), xs, label="PF-x", color="orange", s=0.5)

    ax[1, rx_idx].plot(
        reduced_gt_theta,
        label=f"r{rx_idx} reduced gt theta",
        color="black",
        linestyle="dashed",
        linewidth=1,
    )

    ax[0, rx_idx].set_ylabel("radio phi")
    ax[0, rx_idx].set_title(f"Radio {rx_idx}")
    ax[1, rx_idx].set_xlabel("time step")
    ax[1, rx_idx].set_ylabel("radio theta")
    ax[0, rx_idx].legend()
    ax[1, rx_idx].legend()

    # ax[2, rx_idx].hist(zscores.reshape(-1), bins=25)
fig.suptitle("Single ladies (radios) ParticleFilter")
fig.savefig(f"{output_prefix}_single_ladies_pf.png")

In [ ]:
from spf.rf import pi_norm


# def update_paired(particles, weights, zs, R):
#     # weights *= scipy.stats.norm(hx_pf(particles), R).pdf(z)
#     # print(weights.shape, z.shape, particles.shape)
#     # print(z)
#     for idx in range(weights.shape[0]):
#         for rx_idx in (0, 1):
#             weights[idx] *= theta_phi_to_p(
#                 particles[idx, 0] - offsets[rx_idx], zs[rx_idx], full_p=full_p
#             )
#         # weights[idx] *= scipy.stats.norm(0, 0.0001).pdf(particles[idx, 1])
#     weights += 1.0e-300  # avoid round-off to zero
#     weights /= sum(weights)  # normalize


def update_paired(particles, weights, zs):
    # non-vectorized
    # weights = weights.copy()
    # for idx in range(weights.shape[0]):
    #     weights[idx] *= theta_phi_to_p(particles[idx, 0], z, full_p=full_p)
    # vectorized
    weights *= theta_phi_to_p_vec(
        pi_norm(particles[:, 0] - offsets[0]), zs[0], full_p=full_p
    )
    weights *= theta_phi_to_p_vec(
        pi_norm(particles[:, 0] - offsets[1]), zs[1], full_p=full_p
    )
    # assert np.isclose(_weights, weights).all()
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def fix_particles_paired(particles):
    particles[:, 0] = pi_norm(particles[:, 0])
    return particles


def resample_from_index_paired(particles, weights, indexes):
    # print(indexes)
    particles[:] = particles[indexes]
    # weights[:] = weights[indexes]
    # add noise to the new samples
    noise = np.random.randn(*particles.shape)
    # noise[:, 0] *= 0.01
    # noise[:, 1] *= 0.01
    noise[:, 0] *= 0.1
    noise[:, 1] *= 0.1
    change_mask = indexes[:-1] == indexes[1:]
    particles[1:][change_mask] += noise[1:][change_mask]
    # weights.resize(len(particles))
    weights.fill(1.0 / len(weights))
    # weights[1:][change_mask] *= 0.1
    weights /= sum(weights)  # normalize


def pf_trajectory_for_phi_paired(ds, N=128):
    particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.1), N=N)
    weights = np.ones((N,)) / N
    trajectory = []
    thetas = []
    vs = []
    for idx in range(len(ds)):
        particles = fix_particles_paired(particles)
        predict(particles=particles, dt=0.1)
        particles = fix_particles_paired(particles)
        zs = np.hstack(
            [
                ds[idx][0]["mean_phase_segmentation"],
                ds[idx][1]["mean_phase_segmentation"],
            ]
        )
        update_paired(particles=particles, weights=weights, zs=zs)
        particles = fix_particles_paired(particles)
        # resample if too few effective particles
        if neff(weights) < N / 2:
            # print("RESAMPLE")
            indexes = systematic_resample(weights)
            resample_from_index_paired(particles, weights, indexes)
            # assert np.allclose(weights, 1 / N)
        mu, var = estimate(particles, weights)
        # particles = create_gaussian_particles(mean=mu, std=np.sqrt(var), N=N)
        trajectory.append({"theta": mu[0], "var": var[0]})
        for particle in particles:
            thetas.append((idx, particle[0]))
            vs.append((idx, particle[1]))
    return trajectory, thetas, vs

In [ ]:
traj_paired, theta_paired, vs_paired = pf_trajectory_for_phi_paired(ds)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import reduce_theta_to_positive_y, torch_pi_norm

fig, ax = plt.subplots(2, 1, figsize=(10, 10))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
n = len(traj_paired)
colors = ["blue", "orange"]
for rx_idx in (0, 1):
    ax[0].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=0.1,
        color=colors[rx_idx],
    )
    ax[0].plot(
        ds.ground_truth_phis[rx_idx][:n],
        color=colors[rx_idx],
        label=f"r{rx_idx} perfect phi",
        linestyle="dashed",
    )

ax[1].plot(
    [torch_pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(ds))],
    label="craft gt theta",
    linestyle="dashed",
)

xs = np.array([x["theta"] for x in traj_paired])
stds = np.sqrt(np.array([x["var"] for x in traj_paired]))

ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="PF-std",
    color="red",
    alpha=0.2,
)
ax[1].scatter(range(len(ds)), xs, label="PF-x", color="orange", s=0.5)


ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio 0 & 1")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("Theta between target and receiver craft")

fig.suptitle("When two become one (Sensor fusion) Particle Filter")
fig.savefig(f"{output_prefix}_paired_pf.png")

In [ ]:
ax[2].plot(
    [torch_pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(ds))],
    label="craft gt theta",
)